In [1]:
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import json
from sqlalchemy import create_engine, text
import numpy as np
from PIL import Image
from io import BytesIO
import base64

In [2]:
def get_db(dbname='POS',tablename='Heatmapstore', user="postgres", password='dummy', host='localhost', port=5432, stime=-1, etime=-1):
    engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(user,password,host,port,dbname))
    with engine.connect() as conn:
        query='SELECT * FROM public.\"{}\"'.format(tablename)
        if((len(stime)>=0)and(len(etime)>=0)):
            query+=' WHERE time >= {} AND time <= {}'.format(stime,etime)
        df=pd.read_sql_query(text(query),conn)
        #print(query)
    return df

In [9]:
def data_tables(start_date_str,end_date_str,start_time,end_time):
    # Convert the input strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%d-%m-%Y")
    end_date = datetime.strptime(end_date_str, "%d-%m-%Y")
    
    # Define a timedelta object with a one-day interval
    one_day = timedelta(days=1)

    #table list
    table_list=[]
    
    # Print the dates between the start and end dates
    current_date = start_date
    while current_date <= end_date:
        table_name="metatable"+current_date.strftime("%d%m%Y")
        #print(table_name+",")
        table_list.append(table_name)
        current_date += one_day
    date_time=[{"table":table, "start_time":000000, "end_time":240000} for table in table_list]
    date_time[0]["start_time"]=start_time
    date_time[-1]["end_time"]=end_time
    return date_time

In [11]:
from_date="22-05-2023"
to_date="22-05-2023"
start_time=120000
end_time=130000
dfs=[]
for table_data in data_tables(from_date, to_date,start_time,end_time):
    df=get_db(tablename=table_data['table'],stime=table_data["start_time"],etime=table_data["end_time"])
    dfs.append(df)



11